# Estruturas criptograficas: TP1 problema 1

1. Neste problema foi nos proposto o uso do package Cryptography e do package ascon para criar uma comunicação privada assíncrona em modo “Lightweight Cryptography” entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:

    1. Autenticação do criptograma e dos metadados (associated data) usando Ascon em modo de cifra.
    2. As chaves de cifra, autenticação e os “nounces” são gerados por um gerador pseudo aleatório (PRG) usando o Ascon em modo XOF. As diferentes chaves para inicialização do PRG são inputs do emissor e do receptor.
    3. Para implementar a comunicação cliente-servidor use o package python `asyncio`.

A abordagem

In [1]:
import asyncio
import ascon

async def handle_emitter(key_init_receiver, nonce_init_receiver, reader, writer):
    
    while True:
        # Lê os dados associados
        associateddata = await reader.readuntil(b'_ADEnd_')
        size = int(associateddata.decode()[:-7])
        
        #Lê a mensagem
        ciphertext = await reader.readexactly(size)
        
        #gerar chave e nonce
        key = ascon.hash(key_init_receiver.encode(), variant="Ascon-Xof", hashlength=16)
        nonce = ascon.hash(nonce_init_receiver.encode(), variant="Ascon-Xof", hashlength=16)

        #decifrar a mensagem
        plaintext = ascon.decrypt(key, nonce, associateddata, ciphertext, variant="Ascon-128")
        
        if plaintext is None:
            print("Autentication failed. Message compromised.")
            # writer.close()
        else:
            print(f"Message: {plaintext.decode()}")
            nonce_init_receiver = str(int(nonce_init_receiver) + 1)
            # writer.close()
    
#receiver
async def receiver(key_init_receiver, nonce_init_receiver):
    server = await asyncio.start_server(
        lambda reader, writer: handle_emitter(
            key_init_receiver, nonce_init_receiver, reader, writer), '127.0.0.1', 7777)

    print("Receiver ready...\n")

    async with server:
        await server.serve_forever()


#Emitter
async def send_message( key_init_emitter, nonce_init_emitter, reader, writer):
    
    #establecer concexão
    # reader, writer = await asyncio.open_connection('127.0.0.1', 7777)

    plaintext = input("Enter message: ").encode()
    #associateddata = input("Enter associated data: ")
    
    #dados associados = comprimento do text
    associateddata = (str(len(plaintext)+16) + '_ADEnd_').encode()

    #gerar chaves aliatórias
    key = ascon.hash(key_init_emitter.encode(), variant="Ascon-Xof", hashlength=16)
    nonce = ascon.hash(nonce_init_emitter.encode(), variant="Ascon-Xof", hashlength=16)
    
    #cifrar texto
    ciphertext = ascon.encrypt(
        key, nonce, associateddata, plaintext, variant="Ascon-128")
    
    #anexar dados associados
    message = associateddata + ciphertext

    # Envia a mensagem para o servidor
    writer.write(message)
    await writer.drain()

    # Fecha a conexão
    # writer.close()
    # await writer.wait_closed()
    
async def emitter():
        
    reader, writer = await asyncio.open_connection('127.0.0.1', 7777)
    return reader, writer
 

In [2]:
#Receiver
key_init_receive = "27082001"
nonce_init_receive = "12071972"

asyncio.create_task(receiver(key_init_receive, nonce_init_receive))  # Inicia o servidor

<Task pending name='Task-5' coro=<receiver() running at /tmp/ipykernel_37616/2218372480.py:30>>

Receiver ready...



In [3]:
#Emitter, caso certo

key_init_emitter = "27082001"
nonce_init_emitter = "12071972"

reader, writer = await emitter()
# Envia a mensagem
await send_message( key_init_emitter, nonce_init_emitter, reader, writer)


Message: 270820012708200127082001


In [5]:
#Emitter, key errada

key_init_emitter = "27082001"
nonce_init_emitter = "12071973"

# Envia a mensagem
await send_message(key_init_emitter, nonce_init_emitter, reader, writer) 


Autentication failed. Message compromised.


In [5]:
#Emitter, nonce errado

key_init_emitter = "270801"
nonce_init_emitter = "12071973"

# Envia a mensagem
await send_message(key_init_emitter, nonce_init_emitter) 


Autentication failed. Message compromised.
